### Check an endpoint
This notebook aims to run the pipeline on a given endpoint to check to see if it will be successful. This includes collecting, pipeline and datset stages. It aims to also highlight useful information as a summary as to whether the endpoint would be successful on our platform. it will download all relevant data to do this and hence might be disk intensive. You'll need to provide the following information:
- collection - this is the collection that the dataset belongs to, this can be extracted from the specification but for this notebook we ask to provide it incase you want to test the pipeline on something which isn't being included in the main site right now
- dataset - this is the dataset that the endpoint is meant to provide data for, technically this can be multiple datasets but this this use case it should just be one. It is also the name of the pipeline that is ran on the individual resources that are downloaded from the endpoint. the terms dataset/pipline are often the same
- organisation - the organisation identifier to be used for the endpoint
- endpoint url - the actual url needed for the endpoint
- plugin - often we use plugins to download the data this is only needed for specific endpoints

In [19]:
import os
import pandas as pd
from functions import run_endpoint_workflow
from sqlite_query_functions import DatasetSqlite
from convert_functions import convert_resource
from digital_land.collection import Collection

In [9]:
collection_name = 'conservation-area-collection'
dataset = 'conservation-area'
organisation = 'local-authority-eng:SWK'
endpoint_url = 'https://www.southwark.gov.uk/assets/attach/194104/Conservation-Areas.gpkg'
plugin = None
additonal_column_mappings=None
data_dir = '../data/endpoint_checker'

In [10]:
run_endpoint_workflow(
    collection_name,
    dataset,
    organisation,
    endpoint_url,
    plugin,
    data_dir
)

HTTP Error 404: Not Found
HTTP Error 404: Not Found
HTTP Error 404: Not Found
../data/endpoint_checker/collection
[{'resource': '274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed917a7fe0280aae9b4fa', 'bytes': '192512', 'endpoints': 'ecf6d0fbf0312df98a2f7e8e8b41a65757a5f1eb3ad31859a55c68c6a2300212', 'organisations': 'local-authority-eng:SWK', 'datasets': 'conservation-area', 'start-date': '2023-07-17', 'end-date': '2023-07-17'}]


In [ ]:
# validate endpoint


#### collection log summaries

We need to establish if a resource was downloaded from the endpoint and whether there were any issues during the collection process. Examine the output of the below. There should be one log for the attempt made at downloading from the endpoint. If status code is 200 then the resource was downloaded successfully

In [11]:
collection = Collection(os.path.join(data_dir,'collection'))
collection.load(directory=os.path.join(data_dir,'collection'))
logs = collection.log.entries
logs = pd.DataFrame.from_records(logs)
logs

,bytes,content-type,elapsed,endpoint,resource,status,entry-date,start-date,end-date,exception
0,192512,application/octet-stream,0.411,ecf6d0fbf0312df98a2f7e8e8b41a65757a5f1eb3ad318...,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,200,2023-07-17T21:02:40.975822,,,


### Check unnassigned entiities
This process automatically aims to detect and assign entity numbers where entries are currently unnassigned. Examine the list below to see what (if any) entities have been assigned. if you were to include these in an actual pipeline you would need to update the configuration lookup.csv with these values. It's worth checking they are sensible before this happens 

In [12]:
unassigned_entries = pd.read_csv(os.path.join(data_dir,'var','cache','unassigned-entries.csv'))
if len(unassigned_entries) == 0:
    print('no additional entity numbers where required')
else:
    unassigned_entries

no additional entity numbers where required


#### Check logs collated from the pipeline process
We need to readin in logs and examine to see if the data points were all read in correctly This uses the sqlite database to do so with some custom queries I have wrote. you could directly examine the csvs if the pipeline fails.

first check column mappings to see what columns the pipeline automatically mapped. if this is empty or missing values it's likely to be the reason data isn't appearing at the end.

In [13]:
dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_column_mappings()
results

,end_date,entry_date,field,dataset,start_date,resource,column
0,,2023-07-18T14:29:04Z,geometry,conservation-area,,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,WKT
1,,2023-07-18T14:29:04Z,documentation-url,conservation-area,,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,documentation-url
2,,2023-07-18T14:29:04Z,end-date,conservation-area,,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,end-date
3,,2023-07-18T14:29:04Z,entry-date,conservation-area,,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,entry-date
4,,2023-07-18T14:29:04Z,geometry,conservation-area,,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,geometry
5,,2023-07-18T14:29:04Z,name,conservation-area,,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,name
6,,2023-07-18T14:29:04Z,notes,conservation-area,,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,notes
7,,2023-07-18T14:29:04Z,reference,conservation-area,,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,reference
8,,2023-07-18T14:29:04Z,start-date,conservation-area,,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,start-date


examine the issues logs, we'll look at the types of errors being raised and list all of them. This could be improved in the future by examining the severity

In [15]:
dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_issues_by_type()
results

,issue_type,count
0,default-value,106


In [16]:
results = dataset_db.get_issues()
results

,end_date,entry_date,entry_number,field,issue_type,line_number,dataset,resource,start_date,value
0,,,1,organisation,default-value,2,conservation-area,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,,local-authority-eng:SWK
1,,,1,entry-date,default-value,2,conservation-area,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,,2023-07-17
2,,,2,organisation,default-value,3,conservation-area,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,,local-authority-eng:SWK
3,,,2,entry-date,default-value,3,conservation-area,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,,2023-07-17
4,,,3,organisation,default-value,4,conservation-area,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,,local-authority-eng:SWK
...,...,...,...,...,...,...,...,...,...,...
101,,,51,entry-date,default-value,52,conservation-area,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,,2023-07-17
102,,,52,organisation,default-value,53,conservation-area,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,,local-authority-eng:SWK
103,,,52,entry-date,default-value,53,conservation-area,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,,2023-07-17
104,,,53,organisation,default-value,54,conservation-area,274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed9...,,local-authority-eng:SWK


#### Final dataset comparison against the sqlite database

Below are two tables which show the difference betwen what was provided to us and what is ucrrently in the entity table. It is important to bear in mind that we assign entities automaticallyis process, the table above shows what we have added.

In [17]:
dataset_db = DatasetSqlite(os.path.join(data_dir,'dataset',f'{dataset}.sqlite3'))
results = dataset_db.get_entities()
results

,dataset,end_date,entity,entry_date,geojson,geometry,json,name,organisation_entity,point,prefix,reference,start_date,typology
0,conservation-area,,44002349,2023-07-17,,,"{""documentation-url"": ""https://www.southwark.g...",Bear Gardens,329,,conservation-area,32,,geography
1,conservation-area,,44002350,2023-07-17,,,"{""documentation-url"": ""http://www.southwark.go...",Thrale Street,329,,conservation-area,28,,geography
2,conservation-area,,44002351,2023-07-17,,,"{""documentation-url"": ""https://www.southwark.g...",Union Street,329,,conservation-area,34,,geography
3,conservation-area,,44002352,2023-07-17,,,"{""documentation-url"": ""https://www.southwark.g...",Trinity Church Square,329,,conservation-area,13,,geography
4,conservation-area,,44002353,2023-07-17,,,"{""documentation-url"": ""http://www.southwark.go...",Tooley Street,329,,conservation-area,31,,geography
5,conservation-area,,44002354,2023-07-17,,,"{""documentation-url"": ""http://www.southwark.go...",Tower Bridge,329,,conservation-area,17,,geography
6,conservation-area,,44002355,2023-07-17,,,"{""documentation-url"": ""https://www.southwark.g...",Borough High Street,329,,conservation-area,3,,geography
7,conservation-area,,44002356,2023-07-17,,,"{""documentation-url"": ""http://www.southwark.go...",St Saviours Dock,329,,conservation-area,12,,geography
8,conservation-area,,44002357,2023-07-17,,,"{""documentation-url"": ""http://www.southwark.go...",Wilson Grove,329,,conservation-area,16,,geography
9,conservation-area,,44002358,2023-07-17,,,"{""documentation-url"": ""http://www.southwark.go...",St Marys Rotherhithe,329,,conservation-area,11,,geography


In [20]:
# load in raw resources
collection = Collection(os.path.join(data_dir,'collection'))
collection.load(directory=os.path.join(data_dir,'collection'))
resources = collection.resource.entries
resources

[{'resource': '274345f7c75fbb670408a5d57b5cdd72ba201e2f5b1ed917a7fe0280aae9b4fa',
  'bytes': '192512',
  'organisations': 'local-authority-eng:SWK',
  'datasets': 'conservation-area',
  'endpoints': 'ecf6d0fbf0312df98a2f7e8e8b41a65757a5f1eb3ad31859a55c68c6a2300212',
  'start-date': '2023-07-17',
  'end-date': '2023-07-17'}]

In [26]:
# currently this just reads in the raw resource but in  the future this should check for 
# converted resource first
resource = resources[0]['resource']
resource_path = os.path.join(data_dir,'collection','resource',resource)
try:
    raw_resource = pd.read_csv(resource_path)
except (UnicodeDecodeError,TypeError):
    converted_resource_dir = os.path.join(data_dir,'var','converted_resources')
    converted_resource_path = os.path.join(converted_resource_dir,f'{resource}.csv') 
    if not os.path.exists(converted_resource_path):
        convert_resource(resource,resource_path,converted_resource_dir,dataset)
    raw_resource = pd.read_csv(converted_resource_path)
    

In [27]:
raw_resource

,WKT,reference,name,geometry,documentation-url,notes,start-date,end-date,entry-date
0,"MULTIPOLYGON (((532540.3466 177544.8976,532550...",1,Addington Square,NaN,https://www.southwark.gov.uk/planning-and-buil...,NaN,NaN,NaN,NaN
1,"MULTIPOLYGON (((532822.0968 180500.0017,532804...",3,Borough High Street,NaN,https://www.southwark.gov.uk/planning-and-buil...,NaN,NaN,NaN,NaN
2,"MULTIPOLYGON (((532793.066 176042.5176,532743....",4,Camberwell Grove,NaN,https://www.southwark.gov.uk/planning-and-buil...,NaN,NaN,NaN,NaN
3,"MULTIPOLYGON (((532298.2881 176875.5592,532295...",5,Camberwell New Road,NaN,https://www.southwark.gov.uk/planning-and-buil...,NaN,NaN,NaN,NaN
4,"MULTIPOLYGON (((534817.1213 177474.9076,534801...",6,Caroline Gardens,NaN,https://www.southwark.gov.uk/planning-and-buil...,NaN,NaN,NaN,NaN
5,"MULTIPOLYGON (((533324.1469 173926.7532,533277...",7,Dulwich Village,NaN,https://www.southwark.gov.uk/planning-and-buil...,NaN,NaN,NaN,NaN
6,"MULTIPOLYGON (((533973.297 178021.4012,533968....",8,Glengall Road,NaN,http://www.southwark.gov.uk/planning-and-build...,NaN,NaN,NaN,NaN
7,"MULTIPOLYGON (((531433.2987 178018.0021,531447...",9,Kennington Park Road,NaN,http://www.southwark.gov.uk/planning-and-build...,NaN,NaN,NaN,NaN
8,"MULTIPOLYGON (((533210.4722 176901.0419,533218...",10,Sceaux Gardens,NaN,http://www.southwark.gov.uk/planning-and-build...,NaN,NaN,NaN,NaN
9,"MULTIPOLYGON (((535178.2467 179815.1977,535179...",11,St Marys Rotherhithe,NaN,http://www.southwark.gov.uk/planning-and-build...,NaN,NaN,NaN,NaN
